# Import

In [7]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib
import enchant
import re
import math
from collections import Counter
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import gensim
import string
import fasttext
import fasttext.util
from scipy import spatial
from gensim.parsing.preprocessing import remove_stopwords
import gensim.downloader as api

# Load csv

In [2]:
df = pd.read_csv('datoteka_za_rad.csv')

In [4]:
df

,Body ID,articleBody,Headline,Stance
0,0,A small meteorite crashed into a wooded area i...,"Soldier shot, Parliament locked down after gun...",unrelated
1,0,A small meteorite crashed into a wooded area i...,Tourist dubbed ‘Spider Man’ after spider burro...,unrelated
2,0,A small meteorite crashed into a wooded area i...,Luke Somers 'killed in failed rescue attempt i...,unrelated
3,0,A small meteorite crashed into a wooded area i...,BREAKING: Soldier shot at War Memorial in Ottawa,unrelated
4,0,A small meteorite crashed into a wooded area i...,Giant 8ft 9in catfish weighing 19 stone caught...,unrelated
...,...,...,...,...
49967,2532,"ANN ARBOR, Mich. – A pizza delivery man in Mic...","Pizza delivery man gets tipped more than $2,00...",related
49968,2532,"ANN ARBOR, Mich. – A pizza delivery man in Mic...","Pizza delivery man gets $2,000 tip",related
49969,2532,"ANN ARBOR, Mich. – A pizza delivery man in Mic...","Luckiest Pizza Delivery Guy Ever Gets $2,000 Tip",related
49970,2532,"ANN ARBOR, Mich. – A pizza delivery man in Mic...",Ann Arbor pizza delivery driver surprised with...,related


# Gensim - remove stopwords

In [6]:
#from gensim.parsing.preprocessing import remove_stopwords
text = "Nick likes to play football, however he is not too fond of tennis."
filtered_sentence = remove_stopwords(text)

print(filtered_sentence)

Nick likes play football, fond tennis.


# Metoda = GLOVE model
https://stackoverflow.com/questions/65852710/text-similarity-using-word2vec

In [8]:
#from scipy import spatial
#import gensim.downloader as api

model = api.load("glove-wiki-gigaword-50")

In [9]:
s0 = 'Mark zuckerberg owns the facebook company'
s1 = 'Facebook company ceo is mark zuckerberg'
s2 = 'Microsoft is owned by Bill gates'
s3 = 'How to learn japanese'

def preprocess(s):
    return [i.lower() for i in s.split()]

def get_vector(s):
    return np.sum(np.array([model[i] for i in preprocess(s)]), axis=0)


print('s0 vs s1 ->',1 - spatial.distance.cosine(get_vector(s0), get_vector(s1)))
print('s0 vs s2 ->', 1 - spatial.distance.cosine(get_vector(s0), get_vector(s2)))
print('s0 vs s3 ->', 1 - spatial.distance.cosine(get_vector(s0), get_vector(s3)))

s0 vs s1 -> 0.965923011302948
s0 vs s2 -> 0.8659112453460693
s0 vs s3 -> 0.5877998471260071


# Split text into sentences

In [10]:
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

# Remove stopwords
Prvo pretvori u lower pa onda makni stopwords. Inače ostanu one s velikim početnim slovom.

In [17]:
#remove_stopwords(row['articleBody'].lower())

df.reset_index()

for index, row in df.iterrows():
    for sentence in split_into_sentences(row['articleBody']):
        print(remove_stopwords(sentence.lower()))
        print('\n')
        print(sentence)
        break
    break

small meteorite crashed wooded area nicaragua's capital managua overnight, government said sunday.


A small meteorite crashed into a wooded area in Nicaragua's capital of Managua overnight, the government said Sunday.


# Remove punctuation

In [19]:
a_string = '!hi. wh?at is the weat[h]er lik?e.'

new_string = a_string.translate(str.maketrans('', '', string.punctuation))
print(new_string)

hi what is the weather like


# Split into sentences + remove stopwards + remove punctuation

In [24]:
df.reset_index()

for index, row in df.iterrows():
    for sentence in split_into_sentences(row['articleBody']):
        #remove punctuation
        sentence = sentence.translate(str.maketrans('', '', string.punctuation))
        
        #make lowercase
        sentence = sentence.lower()
        
        #remove stopwords
        sentence = remove_stopwords(sentence)
        
        #print
        print(sentence)
        break
    break

small meteorite crashed wooded area nicaraguas capital managua overnight government said sunday


In [25]:
# in short:

df.reset_index()

for index, row in df.iterrows():
    for sentence in split_into_sentences(row['articleBody']):
        #remove all
        sentence = remove_stopwords(sentence.translate(str.maketrans('', '', string.punctuation)).lower())
        
        #print
        print(sentence)
        break
    break

small meteorite crashed wooded area nicaraguas capital managua overnight government said sunday
